In [ ]:
import pandas as pd
import random

### Get our list of players, 

#load our player stats as dataframes

df_wr = pd.read_csv('/Users/michaelrozewski/Desktop/fantasy prject/wr_stats_and_projections.csv')
df_rb = pd.read_csv('/Users/michaelrozewski/Desktop/fantasy prject/rb_stats_and_projections.csv')
df_te = pd.read_csv('/Users/michaelrozewski/Desktop/fantasy prject/te_stats_and_projections.csv')

# matching column names
df_wr[['2019 RUSH TD','2019 RUSH YDS']] = df_wr[['2019 RUSH TDS', '2019 RUSH YARDS']]

# Concat the dfs together
player_stats = pd.concat([df_wr, df_rb, df_te], axis=0)


# Ensure '2019 REC' and '2019 FPTS' columns are numerical
player_stats[['2019 REC', '2019 FPTS']] = player_stats[['2019 REC', '2019 FPTS']].apply(pd.to_numeric, errors='coerce')

# Drop any rows where conversion to numeric failed and NaNs were introduced
player_stats = player_stats.dropna(subset=['2019 REC', '2019 FPTS'])

# Calculate half_ppr and full_ppr
player_stats['half_ppr'] = player_stats['2019 REC'] * 0.5 + player_stats['2019 FPTS']
player_stats['full_ppr'] = player_stats['2019 REC'] +player_stats['2019 FPTS']



only_fantasy = player_stats[['PLAYER NAME', 'PLAYER POSITION', '2019 FPTS', 'half_ppr', 'full_ppr']]


print(only_fantasy)

In [4]:
#Save locally for Tableau
only_fantasy.to_csv('/Users/michaelrozewski/Desktop/fantasy prject/different_pprs.csv')

In [ ]:
## Taking our player fantasy points, lets graph by position to see how they change under different rule sets
import matplotlib.pyplot as plt
import seaborn as sns

top_50_wr = player_stats[player_stats['PLAYER POSITION'] == 'WR'].nlargest(50, '2019 FPTS')
top_50_rb = player_stats[player_stats['PLAYER POSITION'] == 'RB'].nlargest(50, '2019 FPTS')
top_15_te = player_stats[player_stats['PLAYER POSITION'] == 'TE'].nlargest(15, '2019 FPTS')

plot_data_wr = top_50_wr.melt(id_vars=['PLAYER NAME'], 
                              value_vars=['2019 FPTS', 'half_ppr', 'full_ppr'], 
                              var_name='Scoring Type', value_name='Points')

plot_data_rb = top_50_rb.melt(id_vars=['PLAYER NAME'], 
                              value_vars=['2019 FPTS', 'half_ppr', 'full_ppr'], 
                              var_name='Scoring Type', value_name='Points')

plot_data_te = top_15_te.melt(id_vars=['PLAYER NAME'], 
                              value_vars=['2019 FPTS', 'half_ppr', 'full_ppr'], 
                              var_name='Scoring Type', value_name='Points')

# Plot bar graphs
fig, axs = plt.subplots(3, 1, figsize=(14, 24))

# Plot WRs
sns.barplot(x='PLAYER NAME', y='Points', hue='Scoring Type', data=plot_data_wr, ax=axs[0])
axs[0].set_title('Top 50 Wide Receivers Fantasy Points Comparison')
axs[0].set_xlabel('Player Name')
axs[0].set_ylabel('Fantasy Points')
axs[0].tick_params(axis='x', rotation=90)

# Plot RBs
sns.barplot(x='PLAYER NAME', y='Points', hue='Scoring Type', data=plot_data_rb, ax=axs[1])
axs[1].set_title('Top 50 Running Backs Fantasy Points Comparison')
axs[1].set_xlabel('Player Name')
axs[1].set_ylabel('Fantasy Points')
axs[1].tick_params(axis='x', rotation=90)

# Plot TEs
sns.barplot(x='PLAYER NAME', y='Points', hue='Scoring Type', data=plot_data_te, ax=axs[2])
axs[2].set_title('Top 50 Tight Ends Fantasy Points Comparison')
axs[2].set_xlabel('Player Name')
axs[2].set_ylabel('Fantasy Points')
axs[2].tick_params(axis='x', rotation=90)

# Adjust layout
plt.tight_layout()
plt.show()